#### 지표 UPDATE 
   - 주요 주식시장 지수 : spx, dji, ixix, 코스피, 코스닥은 따로 krx 정보시스템에서 취득
   - 반도체지수, VIX 지수: sox, vix
   - 미국 채권 지수, 한국 채권 지수
   - 원화 환율
   - 선물 : 미국 3대 지수, wti, 달러지수 선물

In [1]:
import datetime
import time
from datetime import date

import pickle

import pandas as pd
import numpy as np

import os
import sys
import shutil

import matplotlib.pyplot as plt

In [2]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

from bs4 import BeautifulSoup as bs
import requests

In [3]:
# !pip install chromedriver_autoinstaller

import chromedriver_autoinstaller

driver_name = chromedriver_autoinstaller.install()  # Check if the current version of chromedriver exists
                                      # and if it doesn't exist, download it automatically,
                                      # then add chromedriver to path
time.sleep(1)

In [4]:
driver = wd.Chrome(service=Service(ChromeDriverManager().install()))

In [5]:
def pick_date_in_calendar(start_date, end_date):
    driver.find_element(By.CLASS_NAME, 'DatePickerWrapper_icon__Qw9f8').click()

    start = driver.find_elements(By.CLASS_NAME, 'NativeDateInput_root__wbgyP')[0]
    end = driver.find_elements(By.CLASS_NAME, 'NativeDateInput_root__wbgyP')[1]

    start_element = start.find_element(By.CSS_SELECTOR, 'input[type=date]')
    end_element = end.find_element(By.CSS_SELECTOR, 'input[type=date]')

    start_element.clear()
    start_element.send_keys(start_date)
    start_element.click()

    end_element.clear()
    end_element.send_keys(end_date)
    end_element.click()

    driver.find_element(By.CLASS_NAME, 'inv-button.HistoryDatePicker_apply-button__fPr_G').click()
    
    end_date_table = datetime.datetime.strptime(start_date, '%Y-%m-%d').date().strftime('%m/%d/%Y')
    css_sel = '#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__lRLYJ > main > div > div:nth-child(5) > div > div > div.border.border-main > div > table > tbody'  
    WebDriverWait(driver, 20).until(EC.text_to_be_present_in_element((By.CSS_SELECTOR, css_sel), end_date_table))
    
    return

In [6]:
 def get_table_df(start_date_range, end_date_range):

    pick_date_in_calendar(start_date_range, end_date_range)

    table_class = 'datatable_table__D_jso datatable_table--border__B_zW0 datatable_table--mobile-basic__W2ilt datatable_table--freeze-column__7YoIE'
    df = pd.read_html(driver.page_source, attrs={"class": table_class}, flavor=["lxml", "bs4"])[0]

    df['Date'] = df['Date'].apply(lambda x : datetime.datetime.strptime(x, "%m/%d/%Y"))
    df = df.sort_values(by='Date').reset_index(drop=True) # ascending order and renumbering index starting 0.
    
    return df

In [7]:
def save_dataframe(df, pkl_name):
    pkl_directory = 'data/common_pkl/'
    df.to_pickle(pkl_directory+pkl_name)
    df.to_csv(pkl_directory+pkl_name.replace('pkl','csv'))
    return

In [8]:
start_date = '2021-12-20'
today = datetime.date.today()
today_str = today.strftime('%Y-%m-%d')

In [ ]:
us_bond_10yr_url = 'https://www.investing.com/rates-bonds/u.s.-10-year-bond-yield-historical-data'
driver.get(us_bond_10yr_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Change %'] = df['Change %'].str[:-1].astype('float')

us_10yr = ['date', 'bond_usa_10', 'open', 'high', 'low', 'bond_usa_10_cr']
df.columns = us_10yr

pkl_name = 'us_10yr_bond.pkl'

save_dataframe(df, pkl_name)

In [ ]:
df.head()

In [ ]:
us_bond_2yr_url = 'https://www.investing.com/rates-bonds/u.s.-2-year-bond-yield-historical-data'
driver.get(us_bond_2yr_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Change %'] = df['Change %'].str[:-1].astype('float')

us_2yr = ['date', 'bond_usa_2', 'open', 'high', 'low', 'bond_usa_2_cr']
df.columns = us_2yr

pkl_name = 'us_2yr_bond.pkl'

save_dataframe(df, pkl_name)

In [ ]:
us_bond_3mon_url = 'https://www.investing.com/rates-bonds/u.s.-3-month-bond-yield-historical-data'
driver.get(us_bond_3mon_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Change %'] = df['Change %'].str[:-1].astype('float')

us_3mon = ['date', 'bond_usa_3m', 'open', 'high', 'low', 'bond_usa_3m_cr']

df.columns = us_3mon

pkl_name = 'us_3mon_bond.pkl'

save_dataframe(df, pkl_name)

In [ ]:
kosdaq_url = 'https://www.investing.com/indices/kosdaq-historical-data'
driver.get(kosdaq_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

kosdaq = ['date', 'kosdaq', 'open', 'high', 'low', 'volume', 'kosdaq_cr']

df.columns = kosdaq

pkl_name = 'kosdaq.pkl'

save_dataframe(df, pkl_name)

In [ ]:
kospi_url = 'https://www.investing.com/indices/kospi-historical-data'
driver.get(kospi_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

kospi = ['date', 'kospi', 'open', 'high', 'low', 'volume', 'kospi_cr']

df.columns = kospi

pkl_name = 'kospi.pkl'

save_dataframe(df, pkl_name)

In [ ]:
snp_futures_url = 'https://www.investing.com/indices/us-spx-500-futures-historical-data'
driver.get(snp_futures_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

snp_future = ['date', 'spx_f', 'open', 'high', 'low', 'volume', 'spx_f_cr']

df.columns = snp_future

pkl_name = 'snp_future.pkl'

save_dataframe(df, pkl_name)

In [ ]:
nas_futures_url = 'https://www.investing.com/indices/nq-100-futures-historical-data'
driver.get(nas_futures_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

ixic_future = ['date', 'ixic_f', 'open', 'high', 'low', 'volume', 'ixic_f_cr']

df.columns = ixic_future

pkl_name = 'ixic_future.pkl'

save_dataframe(df, pkl_name)

In [ ]:
dow_futures_url = 'https://www.investing.com/indices/us-30-futures-historical-data'
driver.get(dow_futures_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

dji_future = ['date', 'dji_f', 'open', 'high', 'low', 'volume', 'dji_f_cr']

df.columns = dji_future

pkl_name = 'dji_future.pkl'

save_dataframe(df, pkl_name)

In [ ]:
wti_futures_url = 'https://www.investing.com/commodities/crude-oil-historical-data'
driver.get(wti_futures_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Vol.'] = df['Vol.'].str[:-1].astype('float')
df['Change %'] = df['Change %'].str[:-1].astype('float')

wti_future = ['date', 'wti', 'open', 'high', 'low', 'volume', 'wti_cr']

df.columns = wti_future

pkl_name = 'wti_future.pkl'

save_dataframe(df, pkl_name)

In [9]:
dollar_index_url = 'https://www.investing.com/indices/usdollar-historical-data'
driver.get(dollar_index_url)
time.sleep(1)

df = get_table_df(start_date, today_str)

df['Change %'] = df['Change %'].str[:-1].astype('float')

dxy_future = ['date', 'dxy', 'open', 'high', 'low', 'volume', 'dxy_cr']

df.columns = dxy_future

pkl_name = 'dxy_future.pkl'

save_dataframe(df, pkl_name)

In [ ]:
driver.close()
driver.quit()